In [2]:
import numpy as np
import cv2 as cv
import os

In [24]:
def markCnt(cnt):
    M = cv.moments(cnt)
    if M['m00']>0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv.putText(img, str(i), (cx-10, cy+6), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        return [cx,cy]
    else:
        return [0,0]
    
def isEllipse(cnt, area):
    if(len(cnt)<=5):
        return False
    else:
        ellipse = cv.fitEllipse(cnt)
        #draw ellipse
        (x, y), (MA, ma), angle = cv.fitEllipse(cnt)
        A = np.pi * MA * ma/4
        if area == 0:
            return False
        if A/area<1.2:
            return True
        else:
            return False
        
def solidityHull(cnt, area):
    hull = cv.convexHull(cnt)
    hull_area = cv.contourArea(hull)
    solidityHull = float(area)/hull_area
    
    #rysowanie
    if solidityHull>0.9:
        print(f'solidityHull:{solidityHull}')
    return solidityCircleHull
        
    
def solidityCircle(cnt, area,sc):
    (x,y),radius = cv.minEnclosingCircle(cnt)
    center = (int(x),int(y))
    solidityCircle = float(area)/(max(radius**2,0.000001) * np.pi)
    
    #rysowanie
    if solidityCircle>sc:
        cv.circle(img,center,int(radius),(0,255,0),2)
#         print(f'solidityCircle:{solidityCircle}')
    return solidityCircle
    
def isCircle(cnt):
    perimeter = cv.arcLength(cnt,True)
    epsilon = 0.02*perimeter
    approx = cv.approxPolyDP(cnt,epsilon,True)
    if len(approx)>5:
        #cv.drawContours(img, contours, i, (0,255,0), 3)
#         print(f'approx:{len(approx)}', end="; ")
#         print(f'isCircle', end="; ")
        return True
    else:
        return False
def dist(x1,y1,x2,y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

def groupWalls(angles, latitude):
    angles = sorted(angles)
    first = angles[0]
    tmp = []
    for angle in angles:
        if angle - first > latitude:
            yield tmp
            tmp = []
        tmp.append(angle)
        first = angle
    yield tmp
    
def groupWallsByAngleDistance(circles, latitudeAngle, latitudeDistance):
    res = []
    
    circles = sorted(circles,key = lambda circle: circle.a)
    while len(circles)>0:
        cands = []
        
        first =circles[0]
        for circle in circles:
            if circle.a - first.a < latitudeAngle:
                cands.append(circle)
                first = circle
        first = cands[0]
        minD = calcMinDist(cands)
        wall = []
        for circle in cands:
            if dist(first.x, first.y, circle.x, circle.y) < minD*latitudeDistance:
                wall.append(circle)
        wallOnlyCnt = []
        for circle in wall:
            wallOnlyCnt.append(circle.i)
            index = None
            for z,c2 in enumerate(circles):
                if c2.i == circle.i:
                    index = z
            circles.pop(index)
        res.append(wallOnlyCnt)
    return res

class Circle:
    def __init__(self,i,x,y,a):
        self.i = i
        self.x = x
        self.y = y
        self.thisDistances = {}
        self.Arr =[]
        self.a = a

In [4]:
def calcDistances(circles):
    for c1 in circles:
        tmp = {}
        for c2 in circles:
            if c1.i != c2.i:
                tmp[c2.i] = dist(c1.x,c1.y,c2.x,c2.y)
        c1.Arr = sorted(tmp.items(), key =
             lambda kv:(kv[1], kv[0]))
        c1.thisDistances = tmp
def calcMinDist(circles):
    distArr = []
    for c in circles:
        for k,v in c.thisDistances.items():
            distArr.append([c.i, k, v])
    distArr = sorted(distArr, key = lambda t:t[2])
#     print(f'min :{distArr[0][0]}, {distArr[0][1]}')
    minDist = distArr[0][2]
    return minDist
def clearDir(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [25]:

walls_by_angle = [] #liczba oczek na ścianach
latitude = 180*0.05 #tolerancja
latitudeDistance = 5 #tolerancja odleglosc oczek na jednej scianie przemnozona razy minimlana odleglosc
aproxCircle = 0.9 #czy jest okragiem dla funkcji solidityCircle
sets = ['set/0','set/1'] #foldery z obrazami

for index,setDir in enumerate(sets):
    clearDir('result/' + str(index+1))
    for fileName in os.listdir(setDir):
        
        #wczytanie, przygotowanie obrazu, kontury
        print(fileName,'----------------')
        f = os.path.join(setDir, fileName)
        if os.path.isfile(f) == False:
            continue
        img = cv.imread(f)
        imgray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        ret, thresh = cv.threshold(imgray, 127, 255, 0)
        contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        angles = []
        circles = []
        #cv.imwrite('thresh.png', thresh)
        for i, cnt in enumerate(contours):
            if i==0:
                continue          
            [cx, cy] = markCnt(cnt)# centroid
            area = cv.contourArea(cnt)# powierzchnia
            cv.drawContours(img, contours, i, (0,255,0), 3)


            #szukanie oczka ilorazie jego powierzchbni i powierzchni elipsy
            if isEllipse(cnt, area):
                cv.ellipse(img,cv.fitEllipse(cnt),(0,255,255),2)
                ellipse = cv.fitEllipse(cnt)
                angle = ellipse[2]
                angles.append(angle)
    #             print('isEllipse--', end="; ")
    #             print(i, angle, ' ')
            markCnt(cnt)   

            #solidity Hull 
            if solidityCircle(cnt, area, aproxCircle)>aproxCircle:
                angle = -1
                angles[-1] = -1

            #szukanie oczka po liczbie wierzcohlkow
            if isCircle(cnt) and isEllipse(cnt, area):
                c = Circle(i, cx, cy, angle)
                circles.append(c)
    #         print('')

        
        
        
        ## PO WYSZUKIWANIU OCZEK
        if len(circles)<2:
            print('liczba oczek:',len(circles))
            res = os.path.join('result/' + str(index+1), fileName)
            cv.imwrite(res, img)
            continue
        calcDistances(circles)

        print('Group by angle')
        tmp = list(groupWalls(angles, latitude))
        print(tmp)
        c = 1
        for g in tmp:
            print('liczba oczek na scianie', c, ': ', len(g))
            c += 1

        print('Group by angle and distance')
        tmp = list(groupWallsByAngleDistance(list(circles), latitude, latitudeDistance))

        c = 1
        for wall in tmp:
            print('liczba oczek na scianie', c, ': ', len(wall), ', kontury:',wall)
            c += 1

        #zapis obrazu
        print('')
        
        res = os.path.join('result/' + str(index), fileName)
        
        cv.imwrite(res, img)

print('koniec') 
# liczba oczek na scianie 1 :  17 ::: [37, 46, 41, 39, 42, 55, 33, 35, 44, 28, 16, 14, 10, 5, 1, 9, 3]
# liczba oczek na scianie 2 :  3 ::: [40, 25, 20]
# liczba oczek na scianie 3 :  12 ::: [30, 34, 24, 31, 27, 26, 22, 52, 47, 56, 54, 51]
# liczba oczek na scianie 4 :  3 ::: [53, 18, 23]
# liczba oczek na scianie 5 :  9 ::: [4, 48, 8, 2, 57, 38, 12, 15, 58]
#

dice.png ----------------
Group by angle
[[-1, -1, -1, -1, -1, -1]]
liczba oczek na scianie 1 :  6
Group by angle and distance
liczba oczek na scianie 1 :  6 , kontury: [3, 4, 5, 6, 7, 8]

dice2.png ----------------
Group by angle
[[154.056640625, 154.1280517578125, 154.58399963378906]]
liczba oczek na scianie 1 :  3
Group by angle and distance
liczba oczek na scianie 1 :  3 , kontury: [2, 3, 1]

dice3.png ----------------
Group by angle
[[17.563140869140625, 19.7202205657959, 21.06193733215332, 22.268312454223633, 25.652530670166016], [88.58885955810547, 89.29593658447266, 90.09275817871094, 90.24081420898438, 90.82770538330078, 91.60245513916016], [154.44015502929688, 157.78089904785156, 160.37338256835938, 162.8089141845703]]
liczba oczek na scianie 1 :  5
liczba oczek na scianie 2 :  6
liczba oczek na scianie 3 :  4
Group by angle and distance
liczba oczek na scianie 1 :  5 , kontury: [8, 6, 5, 4, 1]
liczba oczek na scianie 2 :  6 , kontury: [13, 11, 10, 15, 14, 12]
liczba oczek na

Group by angle
[[-1, -1, -1, -1, -1, -1], [58.5582275390625], [149.7653350830078]]
liczba oczek na scianie 1 :  6
liczba oczek na scianie 2 :  1
liczba oczek na scianie 3 :  1
Group by angle and distance
liczba oczek na scianie 1 :  6 , kontury: [26, 27, 28, 35, 36, 38]

block ----------------
koniec


[1]
